# **1. Tổng quan bài toán**


*   Tập dữ liệu IMDB có 50 nghìn đánh giá phim, từ đó ta để có thể tiến hành Xử lý ngôn ngữ tự nhiên hoặc Phân tích văn bản.
*   Đây là tập dữ liệu phân lớp nhị phân, chứa nhiều dữ liệu hơn đáng kể so với tập dữ liệu chuẩn trước đó. Tập dữ liệu này cung cấp một bộ gồm 25.000 bài đánh giá phim có tính phân cực cao để đào tạo và 25.000 để thử nghiệm. 

*   Thông tin chi tiết về dữ liệu:
http://ai.stanford.edu/~amaas/data/sentiment/

Ở đây, tôi sẽ đào tạo một mô hình để dự đoán xem đánh giá phim IMDB là tích cực hay tiêu cực bằng cách sử dụng BERT trong Tensorflow với TF-Hub.


1.   Input bài toán: IMDB Dataset of 50K Movie Reviews (Large Movie Review Dataset)
2.   Output bài toán: Mô hình BERT dự đoán một đánh giá là tích cực hay tiêu cực.

# **2. Khai báo các thư viện cần thiêt**

In [3]:
!pip install pyspellchecker
!pip install colorama
!pip install simpletransformers
!pip install transformers

In [5]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 
import seaborn as sns
import tensorflow as tf
from datetime import datetime

import os
from pathlib import Path

import plotly.offline as plty
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff

from wordcloud import WordCloud
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from colorama import Fore, Back, Style, init

from tqdm import tqdm
tqdm.pandas()
pd.set_option('display.max_colwidth', None)

from spellchecker import SpellChecker
import spacy
import string, re
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

import transformers
import tensorflow as tf
from tokenizers import BertWordPieceTokenizer

from tensorflow.keras.callbacks import Callback
from sklearn.metrics import accuracy_score, roc_auc_score
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Input, Dropout, Embedding
from tensorflow.keras.layers import LSTM, GRU, Conv1D, SpatialDropout1D

from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import activations
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

import tensorflow.keras.backend as K
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.activations import *
from tensorflow.keras.constraints import *
from tensorflow.keras.initializers import *
from tensorflow.keras.regularizers import *
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

# **3.  Xây dựng dữ liệu**

## **3.1. Load dữ liệu vào DataFrame**

Tập dữ liệu gồm hai trường:
1.   Trường "review" gồm các đoạn văn bản với nội dung đánh giá về một bộ phim.
2.   Trường "sentiment" tương ứng là Label cho đoạn văn bản đó có nội dung Tích cực hay Tiêu cực.

In [6]:
df = pd.read_csv("../input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")

In [12]:
df.info()

In [13]:
df.head(5)

## **3.2. Tiền xử lý dữ liệu**

Có thể nhận thấy dữ liệu ở trường "**sentiment**" đang ở dạng chữ, trong khi đây là dữ liệu dạng nhị phân. Hơn thế nữa trong các bài toán Học máy, chúng ta thường đưa dữ liệu Label ở dạng này về dạng số để dễ xử lý hơn và trong bài toán này cũng vậy.

Vì vậy, ta sẽ Mapping trường này về dạng Numbers, với:


*   1 thể hiện cho nhận xét Review Tích cực.
*   0 thể hiện cho nhận xét Review Tiêu cực.



In [7]:
df['sentiment'] = df['sentiment'].map({'positive' : 1, 'negative' : 0})

In [15]:
df[['review','sentiment']].head(5)

Ta có thể nhận thấy dữ liệu ở trường "**review**" còn khá nhiều ký hiệu `<br />` thể hiện xuống dòng trong HTML, các dấu chấm, dấu phẩy, dấu ngoặc, v.v.. rất nhiều ký tự gây nhiễu và không có giá trị trong mô hình. Các ký hiệu này có thể làm mô hình dự đoán bị sai lệch. 

Mặt khác, các ký tự trong các đoạn văn bản đang ở dạng viết hoa và viết thường khá lộn xộn và cần đưa về một dạng thống nhất gồm toàn các từ ngữ viết thường.

Ngoài ra, ở đây để cẩn thận hơn, chúng ta có thể sử dụng thư viện Spell Checker để check chính tả cho dữ liệu đầu vào.

In [8]:
PUNCT_TO_REMOVE = string.punctuation

def remove_punctuation(text):    
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [9]:
def clean_text(text):
    
    # Lower Casing
    text = text.lower()
    
    # Remove url
    text = remove_urls(text) 
    
    # Remove html tags
    text = remove_html(text)
    
    # Removing @tags
    text = re.sub('@\w*','',text)
    
    # Removing Punctuations
    text = remove_punctuation(text)
    
    # Removing new lines
    text = re.sub('\\n',' ',text)
    
    return text

In [10]:
df["clean_text"] = df["review"].progress_apply(lambda text: clean_text(text))

In [19]:
df[['review','clean_text']].head(2)

## **3.3. Trực quan hoá dữ liệu**

### **3.3.1. Mức độ phổ biến của các từ**

In [20]:
test_string = ' '.join(df.query('sentiment == 0')['clean_text'])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(test_string)
fig = px.imshow(wordcloud)
fig.update_layout(title_text='Độ phổ biến của các từ trong các đánh giá Tiêu cực')

In [21]:
test_string = ' '.join(df.query('sentiment == 1')['clean_text'])
wordcloud = WordCloud(max_font_size=None, background_color='black', collocations=False,
                      width=1200, height=1000).generate(test_string)
fig = px.imshow(wordcloud)
fig.update_layout(title_text='Độ phổ biến của các từ trong các đánh giá Tích cực')

### **3.3.2. Đánh giá quan điểm trên tập dữ liệu**

Ta sẽ áp dụng mô hình VADER (Valence Aware Dictionary for Sentiment Reasoning), một mô hình được sử dụng để phân tích thái cực tình cảm văn bản với cả hai cực (tích cực / tiêu cực) của cảm xúc. 

VADER ánh xạ các đặc điểm từ vựng với cường độ cảm xúc thành một danh sách tạm dịch là **điểm Sentiment**. 

Ví dụ - những từ như "love", "enjoy", "happy", "like" đều thể hiện tình cảm tích cực. Ngoài ra VADER hiểu ngữ cảnh cơ bản của những từ này, chẳng hạn như "did not love" là một câu nói mang tính tiêu cực. Nó cũng hiểu được sự nhấn mạnh của viết hoa và dấu chấm câu, chẳng hạn như "ENJOY"

In [22]:
def polarity(text):
    if type(text) == str:
        return sid.polarity_scores(text)
    else:
        return -1
    
sid = SentimentIntensityAnalyzer()
df["polarity"] = df["clean_text"].progress_apply(polarity)

In [23]:
df["polarity"].head()

#### 3.3.2.1. Quan điểm đánh giá tiêu cực 

In [24]:
neg_pol = [pols['neg'] for pols in df["polarity"] if type(pols) is dict]
neg_pol = list(filter((0.0).__ne__, neg_pol))

fig = go.Figure(go.Histogram(x=neg_pol, marker=dict(
            color='red')
    ))

fig.update_layout(xaxis_title="Xu hướng", title_text="Xu hướng đánh giá tiêu cực", template="simple_white")
fig.show()

Từ biểu đồ trên, chúng ta có thể thấy rằng xu hướng đánh giá tiêu cực trong tập dữ liệu có giá trị thấp, tập trung quanh mức điểm từ 0-0,3. Chứng tỏ dữ liệu không có những đánh giá quá tiêu cực hoặc gay gắt.

#### 3.3.2.3. Quan điểm đánh giá tích cực 

In [25]:
neg_pol = [pols['pos'] for pols in df["polarity"] if type(pols) is dict]
neg_pol = list(filter((0.0).__ne__, neg_pol))

fig = go.Figure(go.Histogram(x=neg_pol, marker=dict(
            color='blue')
    ))

fig.update_layout(xaxis_title="Xu hướng", title_text="Xu hướng đánh giá tích cực", template="simple_white")
fig.show()

Qua biểu đồ trên, kết hợp với phần 3.3.2.1, ta bước đầu rút ra nhận xét rằng hầu hết các đánh giá sẽ đều mang tính Trung lập, dựa vào mức điểm đánh giá thấp của cả Tiêu cực và Tích cực.

#### 3.3.2.2. Quan điểm đánh giá trung lập

In [26]:
neg_pol = [pols['neu'] for pols in df["polarity"] if type(pols) is dict]
neg_pol = list(filter((0.0).__ne__, neg_pol))

fig = go.Figure(go.Histogram(x=neg_pol, marker=dict(
            color='orange')
    ))

fig.update_layout(xaxis_title="Xu hướng", title_text="Xu hướng đánh giá trung lập", template="simple_white")
fig.show()

Từ biểu đồ trên, chúng ta có thể thấy rằng sự phân bố đánh giá trung lập của tập dữ liệu có xu hướng lớn, tập trung chủ yếu từ điểm số 0.7-0.8. Điều này cho thấy rằng các nhận xét có xu hướng rất trung lập và không thiên vị. Đồng thời, dựa vào biểu đồ cũng cho thấy rằng hầu hết các bình luận không có tính quan điểm cá nhân cao và phân cực, có nghĩa là hầu hết các bình luận đều là dữ liệu tốt cho quá trình huấn luyện mô hình.

### **3.3.3. Độ cân bằng dữ liệu**

In [27]:
total_comments = df['sentiment'].count()
neg = df['sentiment'].value_counts().loc[1]
Sentiment = ['Tiêu cực','Tích cực']
count = [neg, total_comments-neg]

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "bar"}, {"type": "pie"}]])
fig.add_trace(go.Bar(x=Sentiment,y=count,text=count, marker_color=['#FF3333', '#0000FF']),
             row=1, col=1)

fig.add_trace(go.Pie(labels=Sentiment, values=count, domain=dict(x=[0.5, 1.0]), marker_colors=['#FF3333', '#0000FF']), 
              row=1, col=2)

fig.update_layout(height=600, width=800, title_text="Đánh giá tích cực và tiêu cực", template='plotly_white')
fig.show()

Các ý kiến về đánh giá Tích cực và Tiêu cực có sự cân đối: 25000 ý kiến tích cực và 25000 ý kiến tiêu cực - với tỉ lệ 50-50, không hề có sự mất cân bằng dữ liệu.

## **3.4. Phân chia tập dữ liệu**

In [11]:
df = df.drop('review', 1)
df.rename(columns = {'clean_text':'review'}, inplace = True)

In [29]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
save_path = '/distilbert_base_uncased/'
if not os.path.exists(save_path):
    os.makedirs(save_path)
tokenizer.save_pretrained(save_path)

Ta phân chia tập dữ liệu thành các bộ: Traning Set, Validation Set và Test Set theo tỉ lệ 8 : 0.4 : 1.6

In [12]:
X, y = df['review'].values,df['sentiment'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=100)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.2, stratify=y_test, random_state=100)

In [31]:
X_train_tokens = tokenizer(list(X_train), padding='max_length', truncation=True, return_tensors="tf")
X_train_token_ids = X_train_tokens['input_ids']

X_test_tokens = tokenizer(list(X_test), padding='max_length', truncation=True, return_tensors="tf")
X_test_token_ids = X_test_tokens['input_ids']

X_val_tokens = tokenizer(list(X_val), padding='max_length', truncation=True, return_tensors="tf")
X_val_token_ids = X_val_tokens['input_ids']

In [32]:
print(X_test_token_ids.shape)
print(X_train_token_ids.shape)

# **4.  Mô hình**

In [33]:
transformer = transformers.TFDistilBertModel.\
    from_pretrained('distilbert-base-uncased')
embed = transformer.weights[0].numpy()

print('Vocab : ', np.shape(embed)[0], 'Hidden States/Embed vector size :', np.shape(embed)[1])

In [34]:
MAX_LEN = 512
BATCH_SIZE = 12
STEPS_PER_EPOCH = X_train_token_ids.shape[0] // BATCH_SIZE

In [35]:
reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss',  
                                    factor=0.3, patience=2, 
                                    verbose=1, mode='auto', 
                                    epsilon=0.0001, cooldown=1, min_lr=0.000001)

## **4.1. BERT**

In [36]:
bert_transformer = transformers.TFDistilBertModel.\
    from_pretrained('distilbert-base-uncased')

In [37]:
y_train = y_train.astype('int32')
y_test = y_test.astype('int32')
y_val = y_val.astype('int32')

In [38]:
def build_bert_model(maxlen=MAX_LEN):
    tf.autograph.experimental.do_not_convert(func=None)
    input_ids = Input(shape=(maxlen,), dtype=tf.int32, name="input_word_ids")
    embeddings = bert_transformer(input_ids)[0]

    cls_token = embeddings[:, 0, :]
    x = Dense(maxlen, activation="relu")(cls_token)
    x = Dropout(0.5)(x)
    out = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_ids, outputs=out)

    model.compile(Adam(learning_rate=1.5e-5), 
                      loss='binary_crossentropy', 
                      metrics=['accuracy'])
    
    return model

In [39]:
model = build_bert_model()

train_history = model.fit(
    X_train_token_ids, y_train,
    steps_per_epoch=STEPS_PER_EPOCH,
    validation_data=(X_val_token_ids, y_val),
    epochs=2
)

In [40]:
y_pred = model.predict(X_test_token_ids)

In [41]:
y_pred = y_pred.reshape(y_pred.shape[0])
y_pred

Ở đây ta chọn Threshold cho việc dự đoán Review là tích cực hay tiêu cực là 0.5: 

*   y_preds > 0.5 tương đương với 1
*   y_preds < 0.5 tương đương với 0



In [42]:
y_preds = y_pred > 0.5
y_preds = np.where(y_preds == True, 1, 0)

In [43]:
accuracy = accuracy_score(y_preds, y_test)
accuracy

## **4.2. XLNet**

In [13]:
train_df = pd.DataFrame({ 'text': list(X_train),'label': y_train,}, columns=['text','label'])

In [14]:
test_df = pd.DataFrame({ 'text': list(X_test),'label': y_test,}, columns=['text','label'])

In [46]:
train_df.info()

In [16]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'sliding_window': True,
    'max_seq_length': 64,
    'num_train_epochs': 1,
    'learning_rate': 0.00001,
    'weight_decay': 0.01,
    'train_batch_size': 128,
    'fp16': True,
    'use_cuda': True,
    'output_dir': '/outputs/',
}

In [17]:
from simpletransformers.classification import ClassificationModel
import logging
import sklearn

logging.basicConfig(level=logging.DEBUG)
transformers_logger = logging.getLogger('transformers')
transformers_logger.setLevel(logging.WARNING)

# We use the XLNet base cased pre-trained model.
model_xlnet = ClassificationModel('xlnet', 'xlnet-base-cased', num_labels=2, args=train_args) 

# Train the model, there is no development or validation set for this dataset 
model_xlnet.train_model(train_df)

# Evaluate the model in terms of accuracy score
result, model_outputs, wrong_predictions = model_xlnet.eval_model(test_df, acc=sklearn.metrics.accuracy_score)

In [1]:
result

# **5.  Kết quả thực nghiệm mô hình**

## **5.1. BERT**

In [59]:
samples = ['This movie was amazingly brilliant.','This movie was amazingly awful.','Maybe they should try to get a better cast next time.','Only the first half of the movie was enjoyable','They could have spent the money better helping people']

In [60]:
def visualize_model_preds(y_pred, indices=[0, 1, 2, 3]):

    for idx, i in enumerate(indices):
        if y_test[i] == 0:
            label = "Non-toxic"
            color = f'{Fore.GREEN}'
            symbol = '\u2714'
        else:
            label = "Toxic"
            color = f'{Fore.RED}'
            symbol = '\u2716'

        print('{}{} {}'.format(color, str(idx+1) + ". " + label, symbol))
        print(f'{Style.RESET_ALL}')

        print(X_test[idx]); print("")
        fig = go.Figure()
        if y_test[i] == 1:
            yl = [1 - y_pred[i], y_pred[i]]
            
        else:
            yl = [1 - y_pred[i], y_pred[i]]

        fig.add_trace(go.Bar(x=['Positive', 'Negative'], y=yl, marker=dict(color=["seagreen", "indianred"])))
        fig.update_traces(name=X_test[idx])
        fig.update_layout(xaxis_title="Labels", yaxis_title="Probability", template="plotly_white", title_text="Predictions for validation comment #{}".format(idx+1))
        fig.show()

In [ ]:
y_pred = model.predict(X_test_token_ids)

## **5.2. XLNet**

In [19]:
samples = ['This movie was amazingly brilliant.','This movie was amazingly awful.','Maybe they should try to get a better cast next time.','Only the first half of the movie was enjoyable','They could have spent the money better helping people']
predictions, _ = model_xlnet.predict(samples)
label_dict = {0: 'negative', 1: 'positive'}
for idx, sample in enumerate(samples):
    print('{} - {}: {}'.format(idx, label_dict[predictions[idx]], sample))

# **6.  Kết luận**

* Trong xử lý ngôn ngữ tự nhiên, việc tiền xử lý dữ liệu, Vector hoá được dữ liệu về dạng thích hợp là quan trọng để có thể áp dụng các mô hình Học máy.
* BERT là một mô hình khá tối ưu, nhưng hạn chế của khi chạy các mô hình NLP là dữ liệu còn tương đối hạn chế và thiếu sự đa dạng. Việc đảm bảo dữ liệu có sự cân bằng, chính xác cũng là một vấn đề đáng lưu ý.
* XLNet là một mô hình cải tiến từ BERT, nó cho thấy khả năng Training nhanh hơn, thậm chí là không cần chia Epoch nhưng vẫn cho kết quả gần như tương đương so với BERT.
